In [1]:
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.colors import Color
import io
import os

def create_watermark(text, page_width, page_height):
    # Create a PDF in memory
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=(page_width, page_height))
    
    # Calculate the font size based on the page size (proportional scaling)
    # You can adjust the scaling factor (like 0.05) to make the text smaller or larger
    font_size = min(page_width, page_height) * 0.05
    
    # Set up watermark text properties with the calculated font size
    can.setFont("Helvetica-Bold", font_size)
    
    # Set the red color with transparency (RGBA: 1 for red, 0 for green, 0 for blue, 0.3 for alpha transparency)
    transparent_red = Color(1, 0, 0, alpha=0.6)
    can.setFillColor(transparent_red)
    
    # Translate to the center of the page, rotate, and draw the text
    can.translate(page_width / 2, page_height / 2)  # Center based on actual page dimensions
    can.rotate(45)
    can.drawCentredString(0, 0, text)  # Draw at (0, 0) after translation
    
    # Save and close the canvas
    can.save()
    
    # Move the pointer to the beginning of the StringIO buffer
    packet.seek(0)
    return packet

# Function to add the watermark to an existing PDF
def add_watermark(input_pdf, watermark_text, output_pdf):
    # Read the input PDF
    pdf_reader = PdfReader(input_pdf)
    pdf_writer = PdfWriter()
    
    # Add watermark to each page
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        
        # Get the dimensions of the page (convert Decimal to float)
        page_width = float(page.mediabox.width)
        page_height = float(page.mediabox.height)
        
        # Create a watermark based on the actual page size
        watermark_packet = create_watermark(watermark_text, page_width, page_height)
        watermark_reader = PdfReader(watermark_packet)
        watermark_page = watermark_reader.pages[0]
        
        # Merge watermark with the page
        page.merge_page(watermark_page)
        pdf_writer.add_page(page)
    
    # Write the result to a new PDF file
    with open(output_pdf, 'wb') as out_file:
        pdf_writer.write(out_file)

# Main function with automatic file name handling
def main():
    # Automatically get the input file name
    files = [f for f in os.listdir() if f.endswith(".pdf")]
    
    if len(files) == 0:
        print("No PDF files found in the current directory.")
        return
    
    input_pdf = files[0]  # Select the first PDF file found in the directory
    print(f"Input PDF file selected: {input_pdf}")
    
    # Automatically generate the output file name
    base_name, _ = os.path.splitext(input_pdf)
    output_pdf = f"{base_name}_watermarked.pdf"
    
    # Ask the user for the watermark text
    watermark_text = input("Enter the watermark text: ")

    # Apply watermark
    add_watermark(input_pdf, watermark_text, output_pdf)
    print(f"Watermark applied successfully to {output_pdf}")

if __name__ == "__main__":
    main()

Input PDF file selected: Testing.pdf
Enter the watermark text: Testing watermark
Watermark applied successfully to Testing_watermarked.pdf
